In [16]:
import torch
import torch.nn as nn

In [34]:
rnn = nn.LSTM(384, 20, 1)
input = torch.randn(384)
embedded = input.view(-1, 1, 384)
h0 = torch.randn(1, 1, 20)
c0 = torch.randn(1, 1, 20)

In [35]:
input.shape

torch.Size([384])

In [36]:
embedded.shape

torch.Size([1, 1, 384])

In [37]:
output, (hn, cn) = rnn(embedded, (h0, c0))

In [38]:
output.shape

torch.Size([1, 1, 20])